# WNTR Chlorine Simulation
In this notebook, we explore **water quality simulations** in WNTR using concepts similar to those covered in Lecture 19.  
We’ll use the `Net3.inp` water network model to demonstrate key examples.

In this exercise, you will:
1. **Define and run** a chlorine simulation  
2. **Plot time series** of a residual chlorine at selected junctions  
5. **Visualize spatial patterns** of residual chlorine across the network


## Imports
Install and import WNTR and additional Python packages that are needed for the tutorial
- Numpy is required to define comparison operators (i.e., np.greater) in queries
- Matplotlib is required to create graphics

In [ ]:
# Install required packages if not already available
try:
    import wntr
except ImportError:
    !pip install wntr
    import wntr  # import again after installation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Units
WNTR uses **SI (International System) units (length in meters, time in seconds, mass in kilograms)**.  See https://usepa.github.io/WNTR/units.html for more details.

That means that water age and reaction rates are reported in **s**.

# Chlorine Simulation 💧

## Import network model

In [ ]:
# Create a WaterNetworkModel from an EPANET INP file
inp = 'networks/Net3.inp'
wn = wntr.network.WaterNetworkModel(inp)

## Define and run chlorine simulation ⚙️

In [ ]:
# Quality = chemical (chlorine)
# wn.options.time.duration = 24*3600
wn.options.time.quality_timestep = 300
wn.options.quality.parameter = 'CHEMICAL'
wn.options.quality.chemical_name = 'Chlorine'

In [ ]:
# UNITS
unitsm = 1000 #kg/cubic meters to mg/l
unitsk = 1/24/60/60 # 1/day to 1/s

In [ ]:
# Set source location, initial quality, and bulk reaction rate constant
wn.options.reaction.bulk_coeff = -0.3*unitsk     # 1/day global bulk decay
wn.options.reaction.wall_coeff = 0.0             # no wall decay
# source
lake = wn.get_node('Lake')
lake.initial_quality = 3.0/unitsm

river = wn.get_node('River')
river.initial_quality = 3.0/unitsm

In [ ]:
lake.initial_quality

In [ ]:
# run simulation
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()           

In [ ]:
# get results
# junctions = wn.junction_name_list
# nodes_to_plot = ['10']  # example node IDs from NET3
nodes_to_plot = ['105', '187', '231','275']  # example node IDs from NET3
# nodes_to_plot = ['Lake']  # example node IDs from NET3
concentration = results.node['quality'][nodes_to_plot]*unitsm
concentration.head()

## Plot chlorine concentration at selected nodes

In [ ]:
time_hours = concentration.index / 3600  # Convert seconds to hours

plt.plot(time_hours, concentration, linewidth = 2, alpha = 0.5)
# Formatting the plot
plt.xlabel('Time (hours)')
plt.ylabel('Chlorine (mg/l)')
plt.title('Chlorine')
plt.legend(nodes_to_plot, loc='best') # labels are assigned based on the order that you plot them
plt.show()

## Spatial plot of chlorine residuals across the network 🌍

In [ ]:
concentration_6 = results.node['quality'].loc[6*3600, :]*unitsm

In [ ]:
wntr.graphics.plot_network(
    wn,
    node_attribute = concentration_6,
    title = 'Chlorine residuals at 6am',
    node_cmap = 'coolwarm_r',
    node_size = 25,
    link_width = 0.5,
    add_colorbar = True
)
plt.show()

In [ ]:
concentration_72 = results.node['quality'].loc[72*3600, :]*unitsm

In [ ]:
wntr.graphics.plot_network(
    wn,
    node_attribute = concentration_72,
    title = 'Chlorine residuals at hr 48',
    node_cmap = 'coolwarm_r',
    node_size = 25,
    link_width = 0.5,
    add_colorbar = True
)
plt.show()

## Plot concentrations at all tanks

In [ ]:
# get tank list
tanks = wn.tank_name_list
tanks

In [ ]:
# get tank concentration
tanks_c = results.node['quality'][tanks]*unitsm

In [ ]:
time_hours = concentration.index / 3600  # Convert seconds to hours

plt.plot(time_hours, tanks_c, linewidth = 2, alpha = 0.5)
# Formatting the plot
plt.xlabel('Time (hours)')
plt.ylabel('Chlorine (mg/l)')
plt.title('Tanks')
plt.legend(tanks, loc='best') # labels are assigned based on the order that you plot them
plt.show()

## Effect of tank mixing model on water quality

We'll compare 'FIFO', 'LIFO', 'MIXED' mixing models

Let's demonstrate using tank 1

In [ ]:
# get current mixing model
i = 0
tank = wn.get_node(tanks[i])

In [ ]:
print(tank.mixing_model)  # mixed is default

In [ ]:
# dir(tank)

In [ ]:
tank.mixing_model = 'MIXED'
print(tank.mixing_model)

In [ ]:
# MIXED
#---------------------------------
# run simulation
sim = wntr.sim.EpanetSimulator(wn)
results_mixed = sim.run_sim()

# get tank water age
tank_mixed = results_mixed.node['quality'][tank.name]*unitsm
time_hours = tank_mixed.index / 3600  # Convert seconds to hour
tank_mixed

In [ ]:
# FIFO
#---------------------------------
# set tank mixing model
tank.mixing_model = 'FIFO'

# run simulation
sim = wntr.sim.EpanetSimulator(wn)
results_fifo = sim.run_sim()

# get tank water age
tank_fifo = results_fifo.node['quality'][tank.name]*unitsm
tank_pressure = results_fifo.node['pressure'][tank.name]/3600
tank_fifo

In [ ]:
# LIFO
#---------------------------------
# set tank mixing model
tank.mixing_model = 'LIFO'

# run simulation
sim = wntr.sim.EpanetSimulator(wn)
results_lifo = sim.run_sim()

# get tank water age
tank_lifo = results_lifo.node['quality'][tank.name]*unitsm
tank_lifo

In [ ]:
# Plot timeseries
plt.figure() 
plt.plot(time_hours, tank_mixed)
plt.plot(time_hours, tank_fifo)
plt.plot(time_hours, tank_lifo)
# Formatting the plot
plt.xlabel('Time (hours)')
plt.ylabel('Water age (hours)')
plt.title('Chlorine Residual Time Series at Tank 1')
plt.legend(['mixed','fifo','lifo'])
plt.grid(True)
plt.show()